# AirBnb NY Listing Price Prediction: ML Best Models

In [0]:
%pip install mlflow==2.11.1
%pip install scikit-learn==1.4.0

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import numpy as np
import mlflow

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

### Split into Train & Test

In [0]:
catalog_ = f"price_prediction"
schema_ = f"ny_listing"
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)

In [0]:
SEED_ = 334

gold_data = spark.sql("SELECT * from gold_data")
train_df, test_df = gold_data.randomSplit([.85, .15], seed = SEED_)
train_df = train_df.toPandas()
test_df = test_df.toPandas()

display(train_df.head(2))
#display(test_df.head(2))

In [0]:
# Convert data into np arrays
xTrain = train_df.iloc[:, 1:-1].to_numpy()
yTrain = train_df.loc[:, 'price'].to_numpy()

xTest = test_df.iloc[:, 1:-1].to_numpy()
yTest = test_df.loc[:, 'price'].to_numpy()

### Train Best Models

In [0]:
### MLFlow Parameters

_mlflow_exp = '/Users/gabriele.albini@databricks.com/ModelServing_mlflow/NY_Price_Listings'

mlflow.autolog()

# Pick run IDs corresponding to the best 2-3 models from the tuning phase
xgb_run_ids = ['9aae6a33db714dc0ad421c61f71ddfc4',
               'd1b7da1ab0f046b79d6173e18f623964']

rf_run_ids = ['2a52b95aa4634158a025173b168ec908',
              '7f579c75821e4116832c7ff1a7d4be6c']

# Pick metrics to be logged
metrics_to_log = {
  'mape': mean_absolute_percentage_error,
  'r2': r2_score,
  'mse': mean_squared_error
}

In [0]:
# Function to Create a model with parameters from an MLFlow runID
def create_model(model_type, run_id):
  client = mlflow.tracking.MlflowClient()
  run = client.get_run(run_id)
  parameters = run.data.params
  if model_type == 'GradientBoostingRegressor':
    _model = GradientBoostingRegressor(
      learning_rate = float(parameters['learning_rate']),
      max_depth = int(parameters['max_depth']) if parameters['max_depth'] else parameters['max_depth'],
      max_features = str(parameters['max_features']) if parameters['max_features'] != 'None' else None,
      n_estimators = int(parameters['n_estimators']) if parameters['n_estimators'] else parameters['n_estimators'],
      random_state = SEED_)
  else:
    _model = RandomForestRegressor(
      max_depth = int(parameters['max_depth']) if parameters['max_depth'] != 'None' else None,
      max_features = str(parameters['max_features']) if parameters['max_features'] != 'None' else None,
      n_estimators = int(parameters['n_estimators']) if parameters['n_estimators'] else parameters['n_estimators'],
      random_state = SEED_)
    
  return _model

# Function to Train the model, logging within a running parent mlflow run
def train_test_model(model, xTrain, yTrain, xTest, yTest, metrics):
  with mlflow.start_run(nested=True) as run:
    trained_ = model.fit(xTrain, yTrain)
    pred_yTrain = trained_.predict(xTrain) 
    pred_yTest = trained_.predict(xTest)
    ## ml flow logging
    signature_ = mlflow.models.infer_signature(model_input=xTrain[:10], model_output=pred_yTrain[:10])
    dataset_train = mlflow.data.load_delta(table_name= str(catalog_+'.'+schema_+'.gold_data'))
    mlflow.log_input(dataset_train, context="training") # Allows to build UC Lineage
    performances_ = {}
    for m_name in metrics.keys():
      _metric = metrics[m_name]
      performances_['train'+m_name] = _metric(yTrain, pred_yTrain)
      performances_['test'+m_name] = _metric(yTest, pred_yTest)
    mlflow.sklearn.log_model(sk_model = model, artifact_path = "model", signature=signature_, input_example=xTest)
    run_id = mlflow.active_run().info.run_id

  return trained_, performances_, run_id

##### XGB Runs

In [0]:
### Start XGB Best Run
with mlflow.start_run(experiment_id = mlflow.set_experiment(_mlflow_exp).experiment_id, run_name = "XGB_best") as run:

  xgb_results = {}
  for _id in xgb_run_ids:
    xgb_model = create_model('GradientBoostingRegressor', _id)
    xgb_best, xgb_performances, xgb_run_id = train_test_model(
      xgb_model,
      xTrain,
      yTrain,
      xTest,
      yTest,
      metrics_to_log)
    xgb_results[xgb_run_id] = (xgb_best, xgb_performances)

print(xgb_results)

##### RF Runs

In [0]:
with mlflow.start_run(experiment_id = mlflow.set_experiment(_mlflow_exp).experiment_id, run_name = "RF_best") as run:

  rf_results = {}
  for _id in rf_run_ids:
    rf_model = create_model('RandomForestRegressor', _id)
    rf_best, rf_performances, rf_run_id = train_test_model(
      rf_model,
      xTrain,
      yTrain,
      xTest,
      yTest,
      metrics_to_log)
    rf_results[rf_run_id] = (rf_best, rf_performances)

print(rf_results)

### Register Final Models on UC

In [0]:
# Register XGB Model
XGB_run_id = 'c84875234e8a49cdac549e991290505c'
catalog = catalog_
schema = schema_
model_name = "NY_Price_Listings_XGB"
mlflow.set_registry_uri("databricks-uc")
mlflow.register_model(
    model_uri="runs:/"+XGB_run_id+"/model",
    name=f"{catalog}.{schema}.{model_name}"
)

In [0]:
# Register RF Model
RF_run_id = 'acc17c307379492fafa66bb2013836ab'
catalog = catalog_
schema = schema_
model_name = "NY_Price_Listings_RF"
mlflow.set_registry_uri("databricks-uc")
mlflow.register_model(
    model_uri="runs:/"+RF_run_id+"/model",
    name=f"{catalog}.{schema}.{model_name}"
)